This script processes the csv files containing tweet messages mentioning current movies in theater (retrieved by the script *pull_twitter_data*).

Several additional inputs were used in this script: 1) a csv file containing daily box office for each movie currently in theater; 2) a csv file containing the release date for each movie; 3) a csv file containing the weighed average of IMDB and Rotten Tomatoes user rating of the movie; 4) a csv file containing the twitter follower count for the top 3 billed actors for each movie.
    
The output of this script is a table containing: name of film, box office on a particular day, tweet movies of the film on this day, how many days has the film been released on this day, whether this day is a weekday or weekend (Friday is considered weekend for this purpose)

In [1]:
# Dependencies
import pandas as pd
from datetime import datetime
from blacklist import blacklist
from movies_to_process import movie_list

In [2]:
def currecyconvert(currencystring):
    s1 = currencystring.split("$")[1]
    s2 = s1.split(",")
    s3 = "".join(s for s in s2)
    money = int(s3)
    return money

In [3]:
boxoffice = pd.read_csv('input/boxoffice.csv').set_index("title")
boxoffice = boxoffice.applymap(currecyconvert)

releasedate = pd.read_csv('input/releasedate.csv').set_index("title")

star = pd.read_csv('input/starmeter.csv').set_index("title")
star = pd.DataFrame(star.sum(axis = 1)).rename(columns={0:"starmeter"})

theaters = pd.read_csv('input/theatercount.csv').set_index("title")

In [4]:
allfilms_df = pd.DataFrame({"day":[], 
                            "daily tweet count":[], 
                            "day of week":[], 
                            "title":[], 
                            "box office":[], 
                            "days since release":[], 
                            "weekend":[],
                            "star":[],
                            "theaters":[]})

In [5]:
for movie in movie_list:
    print(f'processing {movie}')
    df = pd.read_csv(f"rawtweets/{movie}.csv")
    blacklist_flag = []
    converted_time = []
    
    for message in df.text:
        if any(word in message for word in blacklist): #check if the tweet message contains any of the blacklisted words
            blacklist_flag.append("yes")
        else:
            blacklist_flag.append("no")
    
    for raw_time in df.rawtime:
        converted_time.append(datetime.strptime(str(raw_time), "%a %b %d %H:%M:%S %z %Y"))
    
    df['blacklist'] = blacklist_flag
    df['time'] = converted_time
    
    date = pd.to_datetime(df["time"])
    df['day of week'] = date.dt.weekday
    df["day"] = date.dt.day
    df['month'] = date.dt.month
    
    for row_index in range(len(df)):
        if df.loc[row_index,"month"] == 8:
            df.loc[row_index,"day"] = df.loc[row_index,"day"]+31
    del df["rawtime"]    
    del df["month"]
    
    df = df.loc[(df["blacklist"] == "no") , :].reset_index(drop= True)
    del df["blacklist"]
    
    df.to_csv(f"processedtweets/{movie}.csv", index=False, header=True)
    
    grouped = df.groupby(['day'])
    daily_tweet_count = grouped["text"].count()
    day_of_week = grouped["day of week"].mean() #within each group, "day of the week" is the same, so mean = "day of the week" for the group
    tally = pd.DataFrame(data=dict(s1=daily_tweet_count, s2=day_of_week)).reset_index()
    tally = tally.rename(columns={"s1":"daily tweet count", "s2":"day of week"})
    
    tally['title'] = movie
    tally['box office'] = ''
    tally['days since release'] = ''
    tally['weekend'] = ''
    tally['star'] = ''
    tally['theaters'] = ''
    
    firstday = tally["day"].min()
    
    
    # trim out the earliest day in the twitter data because the earliest day does not have a complete day of tweet record
    # trim out any days after day 36 (Sunday August 5th)
    trimmed_tally = tally.loc[(tally["day"]>firstday) & (tally["day"]<=36), :].reset_index(drop = True)
    
    for row_index in range(len(trimmed_tally)):
        if trimmed_tally.loc[row_index, "day of week"] in [0, 1, 2, 3]:
            trimmed_tally.loc[row_index, "weekend"] = 0
        else:
            trimmed_tally.loc[row_index, "weekend"] = 1
    
        current_date = trimmed_tally.loc[row_index,"day"]
        trimmed_tally.loc[row_index,"days since release"] = current_date - releasedate.loc[movie, "Opening Date"]
        trimmed_tally.loc[row_index,"box office"] = boxoffice.loc[movie, str(current_date)]
        trimmed_tally.loc[row_index,"star"] = star.loc[movie, "starmeter"]
        trimmed_tally.loc[row_index,"theaters"] = theaters.loc[movie, str(current_date)]

    allfilms_df = pd.concat([allfilms_df, trimmed_tally]).reset_index(drop = True) 
print('done')
allfilms_df.to_csv(f"analysis_results/all_data.csv", index=False, header=True)

processing RampageMovie
processing InfinityWar
processing Deadpool2
processing AdriftMovie
processing Hereditary
processing Oceans8
processing MrRogersMovie
processing Gotti
processing TagMovie
processing JurassicWorld
processing LeaveNoTrace
processing sanju
processing Sicario
processing UncleDrew
processing TheFirstPurge
processing AntMan
processing SorryToBotherYou
processing Soorma
processing EighthGrade
processing HotelT3
processing SkyscraperMovie
processing Incredibles2
processing Blindspotting
processing MammaMia2
processing TheEqualizer2
processing UnfriendedMovie
processing MissionImpossible
processing TeenTitansGOMovie
processing ChristopherRobin
processing nevergoinback
processing TheSpyWhoDumpedMe
done
